# 1. Import of input data

In [1]:
import nbimporter
import sys
import os
from pathlib import Path

In [2]:
#Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 HeatRecovery.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 HeatRecovery.ipynb executed standalone — using directory: {current_dir}")

Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# If not found, try in the parent levels
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Run the file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions may not be available.")

📁 HeatRecovery.ipynb executed standalone — using directory: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw\Energy_systems
📘 Importing libraries from: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw\Libraries.ipynb
📁 Libraries.ipynb — directory impostata a: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw


# 2. Input parameters

In [3]:
energysystem = pd.read_excel(energysystem_path, sheet_name = 'Heat recovery')

In [4]:
Power = energysystem['Thermal capacity [kW]']
ThermalEnergyRecovered = energysystem['Thermal energy recovered [kWh/year]']
massHeatRecovery = Power * (12 / 450) # Source: Nordictec (450 kW - weight: 12 kg) 
Lifespan = 20 #years

# 3. Life cycle inventories

## 3.1 Module A1-3

In [5]:
HeatRecovery_stainlessSteel = massHeatRecovery

## 3.2 Module A4

In [6]:
HeatRecovery_Truk_16_32 = ((500 + 200)*(massHeatRecovery)/1000)
HeatRecovery_Light_Truk = (50*(massHeatRecovery)/1000)

## 3.3 Module B2

In [7]:
HeatRecovery_Light_Truk_B2 = 50 * (Lifespan/2) * 0.01 * (massHeatRecovery/1000)

## 3.4 Module B4

In [8]:
HeatRecoveryNumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
SFHeatRecovery = (RSP / ReqSL)  # Scaling factor

## 3.5 Module C2

In [9]:
HeatRecovery_Transport_to_treatment_plant = 50 * (massHeatRecovery/1000)

## 3.6 Module C3

In [10]:
HeatRecovery_EOL_stainlessSteel = HeatRecovery_stainlessSteel

## 3.7 Module C4

In [11]:
HeatRecovery_EOL_landfill_stainlessSteel = HeatRecovery_EOL_stainlessSteel * LandfillMetals2

## 3.8 Module D1 (related to the export of secondary materials)

In [12]:
HeatRecovery_EOL_recycling_stainlessSteel = (RecyclingMetals2 * RecyclingEfficiencyMetals * HeatRecovery_EOL_stainlessSteel) - (RecycledContentMetals * HeatRecovery_EOL_stainlessSteel)
HeatRecovery_EOL_primary_stainlessSteel = ((RecycledContentMetals * HeatRecovery_EOL_stainlessSteel) - (RecyclingMetals2 * RecyclingEfficiencyMetals * HeatRecovery_EOL_stainlessSteel))* SRmetals

# 4. Data for export

In [13]:
ThermalEnergyRecovered = np.array([ThermalEnergyRecovered])
ThermalEnergyProduced_HeatRecovery = ThermalEnergyRecovered
